In [1]:
#! Install library
import os
import importlib.util

if importlib.util.find_spec("requests") is None:
	os.system("pip install requests")

if importlib.util.find_spec("beautifulsoup4") is None:
	os.system("pip install beautifulsoup4")

if importlib.util.find_spec("Counter") is None:
	os.system("pip install Counter")

In [2]:
#! Authentication details
import os
import dotenv
import importlib.util

JiraHost = None
ConfluenceHost = None
Username = None
Password = None

if importlib.util.find_spec("google.colab") is not None: ## if using google colab
    if not os.path.exists('.env'):
        from google.colab import files
        uploaded = files.upload()
        file_name = list(uploaded.keys())[0]
        try:
            os.rename(file_name, '.env')
        except:
            pass

try:
    dotenv.load_dotenv('../.env', override=True)

    JiraHost = os.getenv('SECRETS_HOST')
    ConfluenceHost = os.getenv('SECRETS_CONFLUENCE')
    Username = os.getenv('SECRETS_USERNAME')
    Password = os.getenv('SECRETS_PASSWORD')
except:
    display("trouble loading dot env")
    pass

if JiraHost is None or JiraHost == "":
    JiraHost = input("Enter Jira Host")

if ConfluenceHost is None or ConfluenceHost == "":
    ConfluenceHost = input("Enter Confluence Host")

if Username is None or Username == "":
    Username = input("Enter Username")

if Password is None or Password == "":
    Password = input("Enter Password")

display("Jira Host: " + JiraHost)
display("Confluence Host: " + ConfluenceHost)

'Jira Host: https://jira.budgetdirect.com.au/'

'Confluence Host: https://confluence.budgetdirect.com.au/'

In [3]:
#! Functions
import base64
import re
from bs4 import BeautifulSoup
import requests
from collections import Counter
import pandas as pd
import requests
from functools import reduce
import warnings

requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

def _ExpandColumn(self:pd.DataFrame, colName:str, columnsToExpand = [], prefix:str = "Prefix", sentenceCase:bool = True) -> pd.DataFrame:
    if (prefix == "Prefix"):
        prefix = colName + " "
        with warnings.catch_warnings():
          warnings.simplefilter(action='ignore', category=FutureWarning)
          expandedCols = self[colName].apply(lambda x: pd.Series(x).add_prefix(prefix))
        columnsToExpand = [prefix + c for c in columnsToExpand]
    else:
        expandedCols = self[colName].apply(lambda x: pd.Series(x))

    if len(columnsToExpand) > 0:
        expandedCols = expandedCols[columnsToExpand]

    if sentenceCase:
        expandedCols.columns = [fnSentenceCase(c) for c in expandedCols.columns]

    return pd.concat([self.drop(colName, axis=1), expandedCols], axis=1)

pd.DataFrame.expand = _ExpandColumn

def fnSentenceCase(s):
    s = (' '.join(dict.fromkeys(s.split())))  # remove duplicate words
    s = s.replace("0", "") # remove "0"
    s = s.strip()
    return ' '.join([x.capitalize() for x in re.sub(r"([A-Z])", r" \1", s).split()]) # sentence case

def _SentenceCaseColumns(self:pd.DataFrame) -> pd.DataFrame:
    self.columns = [fnSentenceCase(c) for c in self.columns]
    return self

pd.DataFrame.sentence_case_columns = _SentenceCaseColumns

def fnGetDefaultHeaders():
    return {
        "content-type": "application/json",
        "authorization": "Basic " + base64.b64encode((Username + ":" + Password).encode()).decode(),
        "retry-after": "120"
    }

def ApiSpaces(startAt) :
	url = "/rest/api/space"
	headers = fnGetDefaultHeaders()
	params = {
		"type": "global",
		"limit": "50",
		"expand": "permissions",
		"start": str(startAt)
	}
	response = requests.get(ConfluenceHost + url, headers = headers, params = params, verify=False)
	return response.json()

def ApiSpaceContent(key, startAt) :
	url = "/rest/api/space/" + key + "/content"
	headers = fnGetDefaultHeaders()
	params = {
		"depth": "all",
		"limit": "50",
		"expand": "history.contributors.publishers.users",
		"start": str(startAt)
	}
	response = requests.get(ConfluenceHost + url, headers = headers, params = params, verify=False)
	return response.json()

def ApiSpaceWatchers(key, startAt) :
	url = "/rest/api/space/" + key + "/watch"
	headers = fnGetDefaultHeaders()
	params = {
		"limit": "50",
		"start": str(startAt)
	}
	response = requests.get(ConfluenceHost + url, headers = headers, params = params, verify=False)
	return response.json()

def ApiContent(contentId, startAt) :
	url = "/rest/api/content/" + contentId
	headers = fnGetDefaultHeaders()
	params = {
		"limit": "50",
		"start": str(startAt),
		"expand": "history,history.lastUpdated"
	}
	response = requests.get(ConfluenceHost + url, headers = headers, params = params, verify=False)
	return response.json()

def SpacePageCall(key) :
	url = "/spaces/viewspacesummary.action"
	headers = fnGetDefaultHeaders()
	params = {
		"showAllAdmins": "true",
		"key": key
	}
	response = requests.get(ConfluenceHost + url, headers = headers, params = params, verify=False)
	return BeautifulSoup(response.content, "html.parser")

def fnAPI(webRequestDelegate, startAt = 0) -> pd.DataFrame:
    def flatten_reduce_lambda(frm):
        try:
            return list(reduce(lambda x, y: x + y, frm, []))
        except:
            return list(reduce(lambda x, y: x + y, [frm], []))
    def innerGetResults(webRequestDelegate, startAt = 0):
        results = webRequestDelegate(startAt)
        if isinstance(results, dict) and "total" in results and "maxResults" in results:
            if startAt + results["maxResults"] < results["total"]:
                return [results] + innerGetResults(webRequestDelegate, startAt + results["maxResults"])
            else:
                return [results]
        else:
            return [results]
    Source = flatten_reduce_lambda(innerGetResults(webRequestDelegate, startAt))
    df = pd.DataFrame(Source)
    return df

def most_frequent(List):
	try:
		c = Counter(List)
		most_common = [key for key, _ in c.most_common(5)]
		return most_common
	except:
		return ""

In [5]:
import pandas as pd

def fnGetProjectKeys() -> pd.DataFrame:
    def ApiCall(startAt) :
        url = "/rest/api/latest/project"
        headers = fnGetDefaultHeaders()
        params = { }
        response = requests.get(JiraHost + url, headers = headers, params = params, verify=False, allow_redirects=True)
        return response.json()
    df = fnAPI(ApiCall)
    df = df.expand("projectCategory")
    return df[["key"]]

results = fnGetProjectKeys()["key"]

for project in results:
    print (project)

print ("Total =", len(results))

AGP
AGQL
AGG
API
ARC
AWFM
AUTH
AICM
AMP
BCP
BIABAU
BPP
CNC
CAEAI
COS
CAPS
CAD022
CAKM
CCTEL
CQS
CAR
CRV
CCWM
CCA
CR
CCMP
CCM
CCMPRT
CCOS
CL
CDO
CRT
MMACC
CED
AAS
CLAIMS
CAD
CSPD
CARR
CLM
C3PO
CLAIMAR
CAP
CAPA
CLMB
CLMCAT
CCHM
CCMTR
CCRCA
CCON
CCP
CDIG
CHAFS
CK
CMD
CMD1
CMD2
CMD3
CRE
CLARC
CPT
CTS
CAM
CAUAT
CMS
CMR
CSR
CI
CS
CC
CAPI
CDQ
NXGEN
COPS
PUF
CBR
CSI
WOPS
CCV
CSAPICAR
CTMCE
DBA
CTMEO2
CGI
CHMP
ININ
CTMCOMMS
CA
BER
CTMCP
CRM
DATA
CTMDES
CDM
UTL
CTMCC
HML
CTMI
CTMSRV
CTMIT
CTM
CTMMKT
SMP
CTMTIM
CTMTP
CUD
CTMWP
CTMQA
CTP
CDT
QPA
CJOPM
CXD
SEC21
CYBS
CYB
CS22
CS23
DAI
DDSVC1MIP
DPLY
DEPIM
DET
DOQ
DEVOPS
DO
DDOPT1
DDOPT2
DDOPT3
DDSLS1
DDSLS2
DDSLS3
DDSVC1
DDSVC2
DIGI
DIGDEL
DDSEC
DSO
DSD
DWP
DAP
DC1
DC2
TDT
ADMIN
DMP
EPGL
ETP
EPC
EEC
EFIL
EEXP
ERT
FAC
FP
FIN
FPR
FRM
FRAN
GHT
GSP
GCI
GRADPRO
GPDI
GUILDS
HAB
HLT
HDT
HIP
HNC
HNZAMS
HQS
HC
HCPS
HPT
HQQ
HAPB
HUP
IAG
IBMI
LMS
ISA
ICD
ICM
INF19
NET
INFPS
INFP
IPM
INF
IRD
IB
IF
INSPDOC
IAL
IBR
IC
CGEN
INV
IO
ISD
JU73
LEGAL
LIF
PRJLOC
LP
LON